In [6]:
import pandas as pd
import numpy as np
import scipy
from tqdm import tqdm
import pickle
from collections import Counter
from scipy.sparse import csr_matrix
import time
import datetime

In [7]:
transactions = pd.read_csv("transactions_train.csv")

In [8]:
transactions["USER_ID_FIELD"] = transactions['customer_id']
transactions["ITEM_ID_FIELD"] = transactions['article_id']

In [9]:
times = []
for ind, row in tqdm(transactions.iterrows(), total=transactions.shape[0]):
    times.append(time.mktime(datetime.datetime.strptime(row['t_dat'], "%Y-%m-%d").timetuple()))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31788324/31788324 [50:53<00:00, 10409.12it/s]


In [11]:
times = list(map(int, times))

In [12]:
transactions["timestamp"] = times

In [13]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,USER_ID_FIELD,ITEM_ID_FIELD,timestamp
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,1537401600
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,1537401600
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,1537401600
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,1537401600
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,1537401600


In [15]:
transactions = transactions.drop(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'], axis=1)
transactions.head()

,USER_ID_FIELD,ITEM_ID_FIELD,timestamp
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,1537401600
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,1537401600
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,1537401600
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,1537401600
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,1537401600


In [26]:
user_id = []
item_id = []
for ind, row in tqdm(transactions.iterrows(), total=transactions.shape[0]):
     user_id.append(customer2num[row['USER_ID_FIELD']])
     item_id.append(article2num[row['ITEM_ID_FIELD']])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31788324/31788324 [31:20<00:00, 16900.57it/s]


In [28]:
transactions['user_id'] = user_id
transactions['item_id'] = item_id

In [29]:
transactions.head()

,USER_ID_FIELD,ITEM_ID_FIELD,timestamp,user_id,item_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,1537401600,0,0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,1537401600,0,1
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,1537401600,1,2
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,1537401600,1,3
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,1537401600,1,4


In [5]:
time2num = {}
num_time = 0

for ind, row in tqdm(transactions.iterrows()):
    date = row['t_dat']
    if date not in time2num:
        time2num[date] = num_time
        num_time += 1

31788324it [28:18, 18720.65it/s]


In [18]:
#item2time = {}
#for ind, row in tqdm(transactions.iterrows()):
#    item2time[row['article_id']] = row['t_dat']

31788324it [29:54, 17718.89it/s]


In [6]:
user_item_time = {}
for ind, row in tqdm(transactions.iterrows()):
    u_i = (row['customer_id'], row['article_id'])
    t = time.mktime(datetime.datetime.strptime(row['t_dat'], "%Y-%m-%d").timetuple())
    user_item_time[u_i] = int(t)

31788324it [54:52, 9653.81it/s] 


In [22]:
user_item_time

{('000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
  663713001): 1537747200,
 ('000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
  541518023): 1537401600,
 ('00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2',
  505221004): 1537401600,
 ('00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2',
  685687003): 1537401600,
 ('00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2',
  685687004): 1537401600,
 ('00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2',
  685687001): 1537401600,
 ('00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2',
  505221001): 1537401600,
 ('00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4c73235dccbbc132280',
  688873012): 1537401600,
 ('00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4c73235dccbbc132280',
  501323011): 1537401600,
 ('00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4c73235dccbbc132280',
  598859003): 1537401600,
 ('00083cda041544b2fbb0e0d2905ad17da7cf1

In [16]:
#with open('user_item_time.pkl', 'wb') as f:
#    pickle.dump(user_item_time, f)

with open('user_item_time.pkl', 'rb') as f:
    user_item_time = pickle.load(f)

In [125]:
customer2num = {}
num_customer = 0
article2num = {}
num_article = 0

for ind, row in tqdm(transaction_last_month.iterrows()):
    customer_id = row['customer_id']
    if customer_id not in customer2num:
        customer2num[customer_id] = num_customer
        num_customer += 1
        
    article_id = row['article_id']
    if article_id not in article2num:
        article2num[article_id] = num_article
        num_article += 1

9187869it [09:17, 16475.26it/s]


In [18]:
#with open('customer2num.pkl', 'wb') as f:
#    pickle.dump(customer2num, f)
        
with open('customer2num.pkl', 'rb') as f:
    customer2num = pickle.load(f)
    
#with open('article2num.pkl', 'wb') as f:
#    pickle.dump(article2num, f)
        
with open('article2num.pkl', 'rb') as f:
    article2num = pickle.load(f)

In [8]:
user_item_time = {}
for ind, row in tqdm(transactions.iterrows()):
    customer_num = customer2num[row['customer_id']]
    article_num = article2num[row['article_id']]
    user_item_time[(customer_num, article_num)] = time2num[row['t_dat']]

31788324it [33:50, 15654.24it/s]


In [80]:
item_last_time = {}
for (user, item), time in user_item_time.items():
    if item in item_last_time:
        if time > item_last_time[item]:
            item_last_time[item] = time
    else:
        item_last_time[item] = time

In [81]:
for item, time in item_last_time.items():
    item_last_time[item] = int(time / (365 / 2)) + 1

In [85]:
Counter(item_last_time.values())

Counter({2: 18393, 3: 19572, 1: 15039, 4: 36976, 5: 14567})

In [9]:
#with open('user_item_time2.pkl', 'wb') as f:
#    pickle.dump(user_item_time, f)

with open('user_item_time2.pkl', 'rb') as f:
    user_item_time = pickle.load(f)

In [10]:
num2customer = {}
for customer, num in customer2num.items():
    num2customer[num] = customer

num2article = {}
for article, num in article2num.items():
    num2article[num] = article

In [127]:
#user_item_list = list()
#for ind, row in tqdm(transaction.iterrows()):
#    customer_id = row['customer_id']
#    article_id = row['article_id']
#    user_item_list.append((customer2num[customer_id], article2num[article_id]))

9187869it [09:17, 16492.58it/s]


In [6]:
#with open('user_item_list.pkl', 'wb') as f:
#    pickle.dump(user_item_list, f)
        
with open('user_item_list.pkl', 'rb') as f:
    user_item_list = pickle.load(f)

In [7]:
counter = Counter(user_item_list)

In [86]:
data = []
row_ind = []
col_ind = []
ratings = []
for key, value in user_item_time.items():
    row_ind.append(key[0])
    col_ind.append(key[1])
    #rating = int(value / (365 / 2)) + 1
    data.append(item_last_time[key[1]])
    #ratings.append(rating)

In [72]:
Counter(ratings)

Counter({1: 6538009, 3: 5813738, 2: 7816874, 4: 7018798, 5: 119020})

In [87]:
M = len(customer2num)
N = len(article2num)
user_item = csr_matrix((data, (row_ind, col_ind)), shape=(M, N))
#temp = user_item.T @ user_item
#C_sparse = (temp - scipy.sparse.diags((temp).diagonal()))

In [88]:
# initialize a model
model = implicit.als.AlternatingLeastSquares()

# train the model on a sparse matrix of item/user/confidence weights
model.fit(user_item)

  0%|          | 0/15 [00:00<?, ?it/s]

In [28]:
sample_submission = pd.read_csv("sample_submission.csv")

In [61]:
#last_time = transactions[transactions['t_dat'] >= '2020-08-14'] #полтора месяца
popular_items = Counter(transactions['article_id'])
num2submit = 12
most_popular = popular_items.most_common(num2submit)
print(most_popular)
most_popular = [pair[0] for pair in most_popular]

[(706016001, 50287), (706016002, 35043), (372860001, 31718), (610776002, 30199), (759871002, 26329), (464297007, 25025), (372860002, 24458), (610776001, 22451), (399223001, 22236), (706016003, 21241), (720125001, 21063), (156231001, 21013)]


In [89]:
in_history = 0
not_in_history = 0
customer2articles = {}
for ind, row in tqdm(sample_submission.iterrows(), total=sample_submission.shape[0]):
    customer_id = row['customer_id']
    if customer_id in customer2num:
        num = customer2num[customer_id]
        recommends = model.recommend(num, user_item[num], 12)[0].tolist()
        articles = [num2article[item] for item in recommends]
        customer2articles[customer_id] = articles
        in_history += 1
    else:
        customer2articles[customer_id] = most_popular
        not_in_history += 1

1371980it [20:27, 1117.81it/s]


In [90]:
customers_and_articles = [(key, value) for key, value in customer2articles.items()]
customers = [pair[0] for pair in customers_and_articles]
articles = [" ".join(list(map(str, pair[1]))) for pair in customers_and_articles]
submission = pd.DataFrame.from_dict({'customer_id' : customers, 'prediction' : articles})
#submission = submission.drop("Unnamed: 0", axis=1)
submission.to_csv('submission8_als_default_1to5ratings.csv', index=False)

In [92]:
model.recommend(num, user_item[num], 12)

(array([ 1787,  2224,  4859, 64196,  4292,  3376, 67381, 80310, 22594,
        64740,   865, 53711], dtype=int32),
 array([0.00522037, 0.00432137, 0.00393188, 0.00377776, 0.00357945,
        0.0035747 , 0.00344956, 0.00335978, 0.00334101, 0.00314936,
        0.00312614, 0.00310571], dtype=float32))

In [77]:
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568597006 795440003 568601007 507909001 568597...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,699080001 776237020 599580052 759871001 850176...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,699080001 699081001 838055001 713199006 823505...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,730683001 484398001 564786001 757303012 720125...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,918292001 852584001 791587001 912204001 805000...
